Importing Libraries

In [1]:
import numpy as np
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder
smx=SGDClassifier()
lr=LogisticRegression(random_state = 0)
knn = KNeighborsClassifier(n_neighbors=5, metric='euclidean')
dt= DecisionTreeClassifier(criterion = "gini", random_state = 100,
                               max_depth=2, min_samples_leaf=5)
km=KMeans(n_clusters=2)
rf = RandomForestClassifier(n_estimators=100, criterion = 'entropy', random_state = 0)
sv=SVC(kernel='poly')
xgb=XGBClassifier()
gb=GradientBoostingClassifier()
gnb=GaussianNB()

Loading the dataset

In [46]:
import pytreebank
sst = pytreebank.load_sst()

data exploration

In [81]:
rating_count = [0,0,0,0,0]
total_count = 0
char_len = []
word_len = []
for tree in sst['train']:
    total_count += 1
    rating_count[tree.label] += 1
    char_len.append(len(tree.to_lines()[0]))
    word_len.append(len(tree.to_lines()[0].split()))
    if (sst['train'].index(tree) == 0):
        print(tree)
        for node in tree.to_labeled_lines():
            print(node)
        print(tree.to_labeled_lines())
        print(tree.to_lines()[0]) 
        print(tree.label)
print(total_count)
print(rating_count)
print(np.mean(char_len))
print(np.mean(word_len))

(3 (2 (2 The) (2 Rock) )(4 (3 (2 is) (4 (2 destined) (2 (2 (2 (2 (2 to) (2 (2 be) (2 (2 the) (2 (2 21st) (2 (2 (2 Century) (2 's) )(2 (3 new) (2 (2 ``) (2 Conan) )))))))(2 '') )(2 and) )(3 (2 that) (3 (2 he) (3 (2 's) (3 (2 going) (3 (2 to) (4 (3 (2 make) (3 (3 (2 a) (3 splash) )(2 (2 even) (3 greater) )))(2 (2 than) (2 (2 (2 (2 (1 (2 Arnold) (2 Schwarzenegger) )(2 ,) )(2 (2 Jean-Claud) (2 (2 Van) (2 Damme) )))(2 or) )(2 (2 Steven) (2 Segal) ))))))))))))(2 .) ))
(3, "The Rock is destined to be the 21st Century 's new `` Conan '' and that he 's going to make a splash even greater than Arnold Schwarzenegger , Jean-Claud Van Damme or Steven Segal .")
(2, 'The Rock')
(2, 'The')
(2, 'Rock')
(4, "is destined to be the 21st Century 's new `` Conan '' and that he 's going to make a splash even greater than Arnold Schwarzenegger , Jean-Claud Van Damme or Steven Segal .")
(3, "is destined to be the 21st Century 's new `` Conan '' and that he 's going to make a splash even greater than Arnold Sch

In [49]:
import pandas as pd
data_train = [
                (
                    tree.to_lines()[0],
                    tree.label
                )
                for tree in sst['train']
            ]
data_test = [(
  tree.to_lines()[0],
  tree.label
)
for tree in sst['test']
]
x_train=pd.DataFrame(data_train,columns=['sentence','label'])
x_test=pd.DataFrame(data_test,columns=['sentence','label'])
x_train_sent=x_train[['sentence']]
print(x_train_sent)
y_train_ind=x_train['label']
y_train_ind
x_test_sent=x_test[['sentence']]
y_test_ind=x_test['label']
x_test_sent


                                               sentence
0     The Rock is destined to be the 21st Century 's...
1     The gorgeously elaborate continuation of `` Th...
2     Singer/composer Bryan Adams contributes a slew...
3     You 'd think by now America would have had eno...
4                  Yet the act is still charming here .
...                                                 ...
8539                                    A real snooze .
8540                                     No surprises .
8541  We 've seen the hippie-turned-yuppie plot befo...
8542  Her fans walked out muttering words like `` ho...
8543                                In this case zero .

[8544 rows x 1 columns]


,sentence
0,Effective but too-tepid biopic
1,If you sometimes like to go to the movies to h...
2,"Emerges as something rare , an issue movie tha..."
3,The film provides some great insight into the ...
4,Offers that rare combination of entertainment ...
...,...
2205,An imaginative comedy/thriller .
2206,"( A ) rare , beautiful film ."
2207,( An ) hilarious romantic comedy .
2208,Never ( sinks ) into exploitation .


In [36]:
x_train.label

<bound method IndexOpsMixin.value_counts of 0       3
1       4
2       3
3       2
4       3
       ..
8539    0
8540    1
8541    3
8542    0
8543    1
Name: label, Length: 8544, dtype: int64>

Pre-Processing

In [4]:


from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
tokenizer = Tokenizer(num_words=2500, lower=True,split=' ')
tokenizer.fit_on_texts(x_train_sent['sentence'])
tokenizer.fit_on_texts(x_test_sent['sentence'])
# print(len(tokenizer.word_index))
X = tokenizer.texts_to_sequences(list(x_train_sent['sentence']))
x=tokenizer.texts_to_sequences(list(x_test_sent['sentence']))
x=pad_sequences(x)
X = pad_sequences(X)
print(X.size)

393024


BERT embeddings

In [5]:


from sentence_transformers import SentenceTransformer

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
model = SentenceTransformer('bert-base-nli-mean-tokens')
#sentences = ['This framework generates embeddings for each input sentence',  'Sentences are passed as a list of string.', 'The quick brown fox jumps over the lazy dog.']
sentence_embeddings = model.encode(x_train_sent['sentence'])
sentence_embeddings_test=model.encode(x_test_sent['sentence'])

In [9]:
def convert(n):
  if(n>2):
    n=1
  else:
    n=0
  return n

In [10]:
y_train_ind=np.array(y_train_ind)
y_test_ind=np.array(y_test_ind)
y_sst2_t=[]
y_sst2_tes=[]
for i in y_test_ind:
  y_sst2_tes.append(convert(i))
for i in y_train_ind:
  y_sst2_t.append(convert(i))
y_sst2_t=np.array(y_sst2_t)
y_sst2_tes=np.array(y_sst2_tes)
y_sst2_t
y_sst2_t.shape

(8544,)

Neural Network

In [11]:

from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
# sc = StandardScaler()
# X = sc.fit_transform(X)
ohe = OneHotEncoder()
y_test_ind = ohe.fit_transform(y_sst2_tes.reshape(-1,1)).toarray()
y_train_ind = ohe.fit_transform(y_sst2_t.reshape(-1,1)).toarray()
# X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.1)
model = Sequential()
model.add(Dense(128, input_dim=(768), activation='relu'))
# model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(2, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.0001), metrics=['accuracy'])

In [12]:
np.array(sentence_embeddings_test).shape

(2210, 768)

In [13]:
history = model.fit(np.array(sentence_embeddings), y_train_ind, epochs=10, batch_size=32)
y_pred = model.predict(np.array(sentence_embeddings_test))
y_pred=np.array(y_pred)
pred = list()
for i in range(len(y_pred)):
    pred.append(np.argmax(y_pred[i]))
test = list()
for i in range(len(y_test_ind)):
    test.append(np.argmax(y_test_ind[i]))
a = accuracy_score(pred,test)
print('Accuracy is:', a*100)

Epoch 1/10
267/267 [==============================] - 0s 776us/step - loss: 0.4379 - accuracy: 0.7967
Epoch 2/10
267/267 [==============================] - 0s 730us/step - loss: 0.3845 - accuracy: 0.8315
Epoch 3/10
267/267 [==============================] - 0s 687us/step - loss: 0.3633 - accuracy: 0.8378
Epoch 4/10
267/267 [==============================] - 0s 682us/step - loss: 0.3482 - accuracy: 0.8464
Epoch 5/10
267/267 [==============================] - 0s 683us/step - loss: 0.3406 - accuracy: 0.8521
Epoch 6/10
267/267 [==============================] - 0s 702us/step - loss: 0.3280 - accuracy: 0.8555
Epoch 7/10
267/267 [==============================] - 0s 705us/step - loss: 0.3165 - accuracy: 0.8635
Epoch 8/10
267/267 [==============================] - 0s 702us/step - loss: 0.3066 - accuracy: 0.8679
Epoch 9/10
267/267 [==============================] - 0s 697us/step - loss: 0.2974 - accuracy: 0.8750
Epoch 10/10
70/70 [==============================] - 0s 385us/step
Accuracy is: 85

ML CLASSIFIERS

In [14]:
#lr.fit(np.array(sentence_embeddings),np.array(y_train_ind))
#lr.fit(np.array(sentence_embeddings),y_sst2_t)
#y_pred=lr.predict(np.array(sentence_embeddings_test))
#y_pred=lr.predict(np.array(sentence_embeddings_test))
#accuracy_score(np.array(y_test_ind),np.array(y_pred))
#accuracy_score(y_sst2_tes,np.array(y_pred))

#xgb.fit(np.array(sentence_embeddings),y_sst2_t)
#y_pred=xgb.predict(np.array(sentence_embeddings_test))
#accuracy_score(y_sst2_tes,np.array(y_pred))

#gb.fit(np.array(sentence_embeddings),y_sst2_t)
#y_pred=gb.predict(np.array(sentence_embeddings_test))
#accuracy_score(y_sst2_tes,np.array(y_pred))

#knn.fit(np.array(sentence_embeddings),np.array(y_train_ind))

rf.fit(np.array(sentence_embeddings),np.array(y_sst2_t))
y_pred=rf.predict(np.array(sentence_embeddings_test))
accuracy_score(y_sst2_tes,y_pred)



0.8266968325791855